# Assignment 8: Neural Networks

Only use the already imported library `numpy` and the Python standard library. For the evaluation you may also use scikit-learn (`sklearn`) and `matplotlib`. Make sure that the dataset `airfoil_self_noise.csv` is in the same directory as the notebook.

List your team members (name and immatriculation number) and indicate whether you are a B.Sc. Data Science or other group in the following cell:

In [198]:
import numpy as np

def load_dataset(path):
    from sklearn.model_selection import train_test_split
    
    data = np.genfromtxt(path)
    X, y = data[:, :5], data[:, 5]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2020)

    return X_train, X_test, y_train, y_test
    

X_train, X_test, y_train, y_test = load_dataset('airfoil_self_noise.csv')

- Kuang-Yu Li, st169971@stud.uni-stuttgart.de, 3440829 
- Ya-Jen Hsu, st169013@stud.uni-stuttgart.de, 3449448 
- Gabriella Ilena, st169935@stud.uni-stuttgart.de, 3440942

## Task 3: Feedforward Neural Network: Programming

In this task, you will implement a feedforward neural network for regression. The hyperparameters of the model are:
- `input_dim`: The dimension of the input vector.
- `output_dim`: The dimension of the output vector.
- `width`: The dimension of each hidden layer.
- `depth`: The number of hidden layers. For B.Sc. Data Science students, this parameter is constant with a value of 1.
- `learning_rate`: The learning rate for gradient descent.
- `epochs`: The number of epochs/iterations performed during training.

B.Sc. Data Science only have to implement for a single hidden layer, i.e. `depth = 1`. All other students have to implement the network for any `depth >= 1`.

The activation function for each hidden layer is ReLU (g(x) = max(0, x)). The output layer uses the identity as activation, since our objective is regression.

You have to implement the `FeedforwardNeuralNetworkRegressor`.

The `__init__` method initializes the network.
Initialize each weight and bias randomly with a standard Gaussian distribution using the numpy function `numpy.random.normal` with default parameters.

The `fit` method trains the network.
Use backpropagation with gradient descent similar to Task 2.
Use the whole training data set for each training epoch.
Use the mean squared error as loss function.

The `predict` method computes the forward-pass of the network.

Evaluate your classifier on the test data with the mean squared error and compare your results to your linear regression model from assignment 3. Try out different hyper-parameters and compare the results. You may want to normalize your input and output data for better performance.

In [199]:
class FeedforwardNeuralNetworkClassifier(object):
    def __init__(self, input_dim, output_dim, width, depth, learning_rate, epochs):
        # Add your code, such as initialization of weights here.
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.width = width
        self.depth = depth
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.w_in = np.random.randn(width, input_dim)
        self.b_in = np.random.randn(width, 1)
        self.w_hidden = np.random.randn(width, width, depth)
        self.b_hidden = np.random.randn(width, 1, depth)
        self.w_out = np.random.randn(output_dim, width)
        self.b_out = np.random.randn(output_dim, 1)
        print("======INITIAL PARAMETERS======")
        print("input_dim: ", input_dim)
        print("output_dim: ", output_dim)
        print("width: ", width)
        print("depth: ", depth)
        print("learning_rate: ", learning_rate)
        print("epochs: ", epochs)
        # print(self.w_in)
        pass
        
    def fit(self, X, y):
        
        # Implement your training here.
        pass
    
    def predict(self, X):
        # Implement your prediction here.
        print("[predict]X shape:",X.shape)
        Z = np.zeros((width, X.shape[0]), dtype=float)
        y_hat = np.zeros((X.shape[0], self.output_dim), dtype=float)
        print("[predict]Z shape:",Z.shape)
        y_predict = list()
        for i in range (X.shape[0]):
            x = X[i, :]
            # print("[predict] x: ",x)
            x_re = x.reshape(X.shape[1], 1)
            # print("[predict] x: ",x_re)
            # print("[predict] w_in:\n ", self.w_in)
            # print("[predict] b_in:\n", self.b_in)
            z = self.w_in.dot(x_re) + self.b_in
            # Z[:, i] = z;
            a = self.relu(z)
            # print("[predict] z:\n ", z)
            # print("[predict] a:\n ", a)
            for j in range(depth):
                z = self.w_hidden[:,:,j].dot(a) + self.b_hidden[:,:, j]
                a = self.relu(z)
            y_hat[i] = self.w_out.dot(a) + self.b_out
            # print("[predict] y_hat:\n ", y_hat)        
        return y_hat
    
    def relu(self, Z):
        return np.maximum(0, Z)
    
    def forward(self, X):
        assert self.w_in.shape[1] == X.shape[1]
        y_hat = np.zeros((X.shape[0], self.output_dim), dtype=float)
        for i in range (X.shape[0]):
            x = X[i, :]
            x_re = x.reshape(X.shape[1], 1)
            z = self.w_in.dot(x_re) + self.b_in
            a = self.relu(z)
            for j in range(depth):
                z = self.w_hidden[:,:,j].dot(a) + self.b_hidden[:,:, j]
                a = self.relu(z)
            y_hat[i] = self.w_out.dot(a) + self.b_out
        return y_hat



In [200]:
# Implement your training and evaluation here.
print("input shape:", X_train.shape)
print("output shape:", y_train.shape)
input_dim = X_train.shape[1]
output_dim = 1
width = 10
depth = 2
learning_rate = 0.01
epochs = 1
myFNNC = FeedforwardNeuralNetworkClassifier(input_dim, output_dim, width, depth, learning_rate, epochs)

y = myFNNC.predict(X_test)
print(" y:\n ", y)

def test():
    print(X_train[1, :])
    x = X_train[1, :]
    print(x.reshape(5,1))
    one = np.ones(5,dtype=float)
    print(one)
    one_15 = np.ones((1,5),dtype=float)
    print(one_15)
    one_reshape = one.reshape(5,1)
    print(one_reshape)
    pass

input shape: (1352, 5)
output shape: (1352,)
======INITIAL PARAMETERS======
input_dim:  5
output_dim:  1
width:  10
depth:  2
learning_rate:  0.01
epochs:  1
[predict]X shape: (151, 5)
[predict]Z shape: (10, 151)
 y:
  [[ -58038.66695899]
 [-114436.44657179]
 [ -36243.32868744]
 [ -19000.06687592]
 [ -12949.57866087]
 [ -29876.12907329]
 [ -58075.39399206]
 [  -8321.25690676]
 [-141696.89454382]
 [ -46087.66653678]
 [ -45880.41642581]
 [ -46750.61388543]
 [ -57290.05906958]
 [ -18993.63697768]
 [  -6600.95417466]
 [  -5260.52864186]
 [ -12598.39231907]
 [  -8377.38417764]
 [ -24236.89277995]
 [ -23511.45591059]
 [ -71844.95515742]
 [ -37153.40767329]
 [ -46115.87368484]
 [ -57656.52785211]
 [  -5288.78882095]
 [ -29188.20642662]
 [ -29872.78182069]
 [ -37229.47925308]
 [  -5303.14428404]
 [  -4170.1915388 ]
 [ -91882.61355076]
 [  -8413.43895779]
 [ -29847.44524832]
 [ -15743.81175681]
 [-142617.63087221]
 [ -23474.1502395 ]
 [ -36894.57585096]
 [ -24243.27852181]
 [  -2822.15206233]
 